In [156]:
import pandas as pd
import numpy as np
from collections import Counter

import mlflow
from mlflow.models import infer_signature

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek

from xgboost import XGBClassifier, plot_importance

import hyperopt as hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

## Load data

In [157]:
df = pd.read_csv("data/customer_churn_telecom_services.csv")
df["TotalCharges"] = df["TotalCharges"].fillna(df["tenure"]*df["MonthlyCharges"])

## Churn Prediction

* Churn prediction modelling can be an important tool for any company that aims to maximize customer retention, thus increase revenues and profitability.
* Predicting churn and successfully lowering it by a relatively small fraction, ensure increase in profits especially when it comes down to acquiring new customers. This can be
better modelled by using numbers such as Customer Lifetime Value (CLTV), Customer Acquisition Cost (CAC), Churn rate and Retention rate.

In addition, having historic data such as the purchase history (membership or upgrades), engagemnt data with the product/service sentiment metrics (complaints, feedback) etc can provide event better insights compared to using static data. 

## Use Case

* The dataset provides a plethora of data with regards to telecom customers. The data are static and it doesnt include any history data.
* The objective is to use the dataset and 'profile' the type of customer who is more likely to churn.

## Data Split

In [158]:
categ_feat = df.select_dtypes(include="object").columns.tolist()
categ_feat.remove("Churn")

df = pd.get_dummies(df, columns=categ_feat, drop_first=True)
df["Churn"] = df["Churn"].map({"No": 0, "Yes": 1})

#### Generate synthetic data

In [159]:
X = df.drop('Churn', axis=1)
y = df['Churn']

# sm = SMOTE(random_state=3)
# ada = ADASYN(random_state=3)
smtom = SMOTETomek(random_state=3)

# X, y = sm.fit_resample(X, y)
# X, y = ada.fit_resample(X, y)
X, y = smtom.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({0: 4753, 1: 4753})


In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

## Hyperparameter tuning - MLflow experiments

In [161]:
# mlflow server --host 127.0.0.1 --port 8080

In [162]:
mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment("Churn prediction 2")

2025/03/01 17:05:39 INFO mlflow.tracking.fluent: Experiment with name 'Churn prediction 2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/235808196041909500', creation_time=1740848739362, experiment_id='235808196041909500', last_update_time=1740848739362, lifecycle_stage='active', name='Churn prediction 2', tags={}>

In [163]:
# hyperparameter space
space={
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10)
    }

In [164]:
signature = infer_signature(X_train, y_train)

/home/pj00/anaconda3/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [165]:
def hyperparameter_tuning(space):
    model = XGBClassifier(n_estimators = int(space['n_estimators']), 
                          max_depth = int(space['max_depth']), 
                          gamma = space['gamma'],
                          reg_alpha = int(space['reg_alpha']), 
                          min_child_weight=space['min_child_weight'],
                          colsample_bytree=space['colsample_bytree'],
                          eval_metric=["rmse"],
                          early_stopping_rounds=10,)
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    with mlflow.start_run(nested=True):
        model.fit(X_train, 
                y_train,
                eval_set=evaluation,
                verbose=False)

        pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, pred>0.5)

        # Log parameters and results
        mlflow.log_params(space)
        mlflow.log_metric("accuracy", accuracy)

        # Log model
        mlflow.xgboost.log_model(model, "model", signature=signature)

        print ("SCORE:", accuracy)
        return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [166]:
with mlflow.start_run():
    trials = Trials()
    best = fmin(fn=hyperparameter_tuning,
                space=space,
                algo=tpe.suggest,
                max_evals=30,
                trials=trials)
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("accuracy", -best_run["loss"])

    # log bets model
    mlflow.xgboost.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

/home/pj00/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:05:44] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)



SCORE:                                                
0.8175604626708728                                    
🏃 View run indecisive-toad-959 at: http://localhost:8080/#/experiments/235808196041909500/runs/1640800987f348f783401d5dfd23ae25

🧪 View experiment at: http://localhost:8080/#/experiments/235808196041909500

  3%|▎         | 1/30 [00:04<01:56,  4.03s/trial, best loss: -0.8175604626708728]

/home/pj00/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:05:48] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)



SCORE:                                                                           
0.8138801261829653                                                               
🏃 View run bald-shrimp-547 at: http://localhost:8080/#/experiments/235808196041909500/runs/0014282137b04dc89f3f26e92ac697ee

🧪 View experiment at: http://localhost:8080/#/experiments/235808196041909500    

  7%|▋         | 2/30 [00:07<01:40,  3.57s/trial, best loss: -0.8175604626708728]

...

SCORE:                                                                            
0.8228180862250263                                                                
🏃 View run enthused-flea-531 at: http://localhost:8080/#/experiments/235808196041909500/runs/c51e2fe3095749cf821379a200b0de42

🧪 View experiment at: http://localhost:8080/#/experiments/235808196041909500     

 97%|█████████▋| 29/30 [01:33<00:03,  3.31s/trial, best loss: -0.8333333333333334]

/home/pj00/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:07:18] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)



SCORE:                                                                            
0.823869610935857                                                                 
🏃 View run flawless-asp-594 at: http://localhost:8080/#/experiments/235808196041909500/runs/24d1b51da1614bc597e6accf067ed655

🧪 View experiment at: http://localhost:8080/#/experiments/235808196041909500     

100%|██████████| 30/30 [01:36<00:00,  3.21s/trial, best loss: -0.8333333333333334]


/home/pj00/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:07:20] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Best parameters: {'colsample_bytree': 0.9959164208865653, 'gamma': 1.359593592969206, 'max_depth': 12.0, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'reg_alpha': 66.0, 'reg_lambda': 0.41252077656727526}
Best eval rmse: -0.8333333333333334
🏃 View run luminous-wasp-592 at: http://localhost:8080/#/experiments/235808196041909500/runs/256c5f7592f4420396fc829c4d105093
🧪 View experiment at: http://localhost:8080/#/experiments/235808196041909500


## Load the best model artifact

In [140]:
# download model artifact directory
mlflow.artifacts.download_artifacts("mlflow-artifacts:/503687084130154334/258f3da1ff74463fa79355c959d4b8f5/artifacts/model/model.xgb", dst_path="best_model")
print("Done")

Done


In [141]:
# Load model
model = XGBClassifier()
model.load_model("./best_model/model.xgb")

In [154]:
predict_y = model.predict(X_test)
accuracy = accuracy_score(y_test, predict_y>0.5)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8268625393494229


In [155]:
model.predict(np.expand_dims(np.array(X_test.loc[0, :]), 0))

array([1])